# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
# import numpy and pandas

import pandas as pd
import numpy as np
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv(r'C:\Users\Daniel Carvalho\Desktop\DataAnalysis\WEEK5\DAY4 - Hypothesis\labs\lab-two-sample-hyp-test\pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:

legendary = pokemon[pokemon['Legendary'] ==True].copy()
non_leg = pokemon[pokemon['Legendary'] ==False].copy()

print(legendary['Name'].count())
print(non_leg['Name'].count())

65
735


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:
print(legendary['Total'].mean(),legendary['Total'].std())
print(non_leg['Total'].mean(),non_leg['Total'].std())


637.3846153846154 60.93738905315346
417.21360544217686 106.76041745713022


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:

alpha = 0.05

st.ttest_ind(legendary['Total'], non_leg['Total'], equal_var= False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957444e-47)

What do you conclude from this test? Write your conclusions below.

In [8]:
# Your conclusions here:

9.357954335957444e-47 < alpha

# In this case we cannot disprove that there is a significant difference between both

True

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [16]:
# Your code here:

types = pokemon['Type 1'].value_counts()
types

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [23]:
# Your code here:

water = pokemon[(pokemon['Type 1'] == 'Water')]
not_water= pokemon[(pokemon['Type 1'] != 'Water')]

print(water['Total'].mean())
print(water['Total'].std())

print(not_water['Total'].mean())
print(not_water['Total'].std())


430.45535714285717
113.18826606431458
435.85901162790697
121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [24]:
# Your code here:

alpha = 0.05

st.ttest_ind(water['Total'], not_water['Total'], equal_var= True)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [25]:
# Your conclusions here:

0.6587140317488793 < alpha

# we keep believing that water pokemon maybe stronger on average then the rest

False

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [30]:
# Your code here:

# H0 : mu defense scores = mu attack scores
# H1 : mu defense scores != mu attack scores

alpha = 0.05

print(st.ttest_rel(pokemon['Defense'], pokemon['Attack'], alternative='less'))

8.570151739679279e-06 < alpha

Ttest_relResult(statistic=-4.325566393330478, pvalue=8.570151739679279e-06)


True

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
#We can say they are different


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [32]:
# Your code here:

alpha = 0.05

print(st.ttest_rel(pokemon['Sp. Def'], pokemon['Sp. Atk'], alternative='less'))

0.1966842998774061 < alpha

Ttest_relResult(statistic=-0.853986188453353, pvalue=0.1966842998774061)


False

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

#We cannot disprove that they are not different

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [34]:
# Your code here:
alpha = 0.05

print(st.ttest_1samp(pokemon['Defense'],0, alternative='less'))
print(st.ttest_1samp(pokemon['Attack'],0, alternative='less'))

# It is not true.

Ttest_1sampResult(statistic=66.97712771588154, pvalue=1.0)
Ttest_1sampResult(statistic=68.84393492906322, pvalue=1.0)


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

